# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 18 2022 3:27PM,248903,22,630935,"NBTY Global, Inc.",Released
1,Oct 18 2022 3:20PM,248902,20,8663636,"ACI Healthcare USA, Inc.",Released
2,Oct 18 2022 3:19PM,248901,10,8664175,Bio-PRF,Executing
3,Oct 18 2022 2:50PM,248899,10,8663655,Yusen – 3D Matrix,Released
4,Oct 18 2022 2:48PM,248898,10,8663654,Yusen – 3D Matrix,Released
5,Oct 18 2022 2:38PM,248893,10,Eme-105114,Emerginnova,Released
6,Oct 18 2022 2:38PM,248893,10,Enova-10825,Emerginnova,Released
7,Oct 18 2022 2:33PM,248892,10,SO94424,"Senseonics, Incorporated",Executing
8,Oct 18 2022 2:32PM,248891,10,0086086335,ISDIN Corporation,Executing
9,Oct 18 2022 2:32PM,248891,10,0086086336,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,248898,Released,1
33,248899,Released,1
34,248901,Executing,1
35,248902,Released,1
36,248903,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248898,NaN,NaN,1.0
248899,NaN,NaN,1.0
248901,NaN,1.0,NaN
248902,NaN,NaN,1.0
248903,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248836,40.0,0.0,5.0
248837,0.0,5.0,32.0
248845,0.0,0.0,3.0
248846,0.0,0.0,18.0
248850,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248836,40,0,5
248837,0,5,32
248845,0,0,3
248846,0,0,18
248850,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248836,40,0,5
1,248837,0,5,32
2,248845,0,0,3
3,248846,0,0,18
4,248850,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248836,40,,5
1,248837,,5,32
2,248845,,,3
3,248846,,,18
4,248850,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc."
1,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc."
2,Oct 18 2022 3:19PM,248901,10,Bio-PRF
3,Oct 18 2022 2:50PM,248899,10,Yusen – 3D Matrix
4,Oct 18 2022 2:48PM,248898,10,Yusen – 3D Matrix
5,Oct 18 2022 2:38PM,248893,10,Emerginnova
7,Oct 18 2022 2:33PM,248892,10,"Senseonics, Incorporated"
8,Oct 18 2022 2:32PM,248891,10,ISDIN Corporation
43,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc."
46,Oct 18 2022 2:19PM,248889,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc.",,,1
1,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc.",,,1
2,Oct 18 2022 3:19PM,248901,10,Bio-PRF,,1,
3,Oct 18 2022 2:50PM,248899,10,Yusen – 3D Matrix,,,1
4,Oct 18 2022 2:48PM,248898,10,Yusen – 3D Matrix,,,1
5,Oct 18 2022 2:38PM,248893,10,Emerginnova,,,2
6,Oct 18 2022 2:33PM,248892,10,"Senseonics, Incorporated",,1,
7,Oct 18 2022 2:32PM,248891,10,ISDIN Corporation,,30,5
8,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",,5,1
9,Oct 18 2022 2:19PM,248889,10,Methapharm-G,,5,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc.",1,,
1,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc.",1,,
2,Oct 18 2022 3:19PM,248901,10,Bio-PRF,,1,
3,Oct 18 2022 2:50PM,248899,10,Yusen – 3D Matrix,1,,
4,Oct 18 2022 2:48PM,248898,10,Yusen – 3D Matrix,1,,
5,Oct 18 2022 2:38PM,248893,10,Emerginnova,2,,
6,Oct 18 2022 2:33PM,248892,10,"Senseonics, Incorporated",,1,
7,Oct 18 2022 2:32PM,248891,10,ISDIN Corporation,5,30,
8,Oct 18 2022 2:19PM,248889,10,"Methapharm, Inc.",1,5,
9,Oct 18 2022 2:19PM,248889,10,Methapharm-G,1,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc.",1,,
1,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc.",1,,
2,Oct 18 2022 3:19PM,248901,10,Bio-PRF,,1,
3,Oct 18 2022 2:50PM,248899,10,Yusen – 3D Matrix,1,,
4,Oct 18 2022 2:48PM,248898,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Oct 18 2022 3:19PM,248901,10,Bio-PRF,NaN,1.0,NaN
3,Oct 18 2022 2:50PM,248899,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Oct 18 2022 2:48PM,248898,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 18 2022 3:27PM,248903,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Oct 18 2022 3:20PM,248902,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Oct 18 2022 3:19PM,248901,10,Bio-PRF,0.0,1.0,0.0
3,Oct 18 2022 2:50PM,248899,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Oct 18 2022 2:48PM,248898,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3733285,26.0,50.0,0.0
15,995364,58.0,5.0,40.0
19,1493165,14.0,15.0,5.0
20,248902,1.0,0.0,0.0
22,995507,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3733285,26.0,50.0,0.0
1,15,995364,58.0,5.0,40.0
2,19,1493165,14.0,15.0,5.0
3,20,248902,1.0,0.0,0.0
4,22,995507,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,26.0,50.0,0.0
1,15,58.0,5.0,40.0
2,19,14.0,15.0,5.0
3,20,1.0,0.0,0.0
4,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,26.0
1,15,Released,58.0
2,19,Released,14.0
3,20,Released,1.0
4,22,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,22
Status,,,,,
Completed,0.0,40.0,5.0,0.0,0.0
Executing,50.0,5.0,15.0,0.0,0.0
Released,26.0,58.0,14.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,22
0,Completed,0.0,40.0,5.0,0.0,0.0
1,Executing,50.0,5.0,15.0,0.0,0.0
2,Released,26.0,58.0,14.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,22
0,Completed,0.0,40.0,5.0,0.0,0.0
1,Executing,50.0,5.0,15.0,0.0,0.0
2,Released,26.0,58.0,14.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()